In [160]:
import csv
import numpy as np

bad_val = np.float64(-9.99000000e+02)

In [142]:

test = csv.reader(open('test.csv', 'r+'), delimiter=',')

train = csv.reader(open('train.csv', 'r+'), delimiter=',')

print(train.line_num)

0


In [143]:
(test_header, test_index, test_letters, test_data) = create_f64_arrays(test, 568239-1)


In [159]:
print(test_data[0,0:])

[ -9.99000000e+02   7.95890000e+01   2.39160000e+01   3.03600000e+00
  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02   9.03000000e-01
   3.03600000e+00   5.60180000e+01   1.53600000e+00  -1.40400000e+00
  -9.99000000e+02   2.20880000e+01  -5.40000000e-01  -6.09000000e-01
   3.39300000e+01  -5.04000000e-01  -1.51100000e+00   4.85090000e+01
   2.02200000e+00   9.85560000e+01   0.00000000e+00  -9.99000000e+02
  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02
  -9.99000000e+02   0.00000000e+00]


In [ ]:
print(test_indices)

In [22]:
##load files

c1 = []
c2 = []
c3 = []


    
    #skip first line
print(len(next(test)))
next(train)

#for row in test:
    #print(row[0])
#        c1.append(row[0])
#        c2.append(row[1])
#        c3.append(row[2])
        
        

32


['Id',
 'Prediction',
 'DER_mass_MMC',
 'DER_mass_transverse_met_lep',
 'DER_mass_vis',
 'DER_pt_h',
 'DER_deltaeta_jet_jet',
 'DER_mass_jet_jet',
 'DER_prodeta_jet_jet',
 'DER_deltar_tau_lep',
 'DER_pt_tot',
 'DER_sum_pt',
 'DER_pt_ratio_lep_tau',
 'DER_met_phi_centrality',
 'DER_lep_eta_centrality',
 'PRI_tau_pt',
 'PRI_tau_eta',
 'PRI_tau_phi',
 'PRI_lep_pt',
 'PRI_lep_eta',
 'PRI_lep_phi',
 'PRI_met',
 'PRI_met_phi',
 'PRI_met_sumet',
 'PRI_jet_num',
 'PRI_jet_leading_pt',
 'PRI_jet_leading_eta',
 'PRI_jet_leading_phi',
 'PRI_jet_subleading_pt',
 'PRI_jet_subleading_eta',
 'PRI_jet_subleading_phi',
 'PRI_jet_all_pt']

In [31]:

#print(np.average(np.float64(c2)))
    
        
print(c2[:20])

[]


In [140]:
#clean -999 values
#def clean_table(table):


    
    
    
def create_f64_arrays(csv, size):
    header = next(csv)
    
    index = np.empty(size, dtype = np.int64)
    letters = np.empty(size, dtype = np.string_)
    ret = np.empty([size, len(header)-2], dtype = np.float64)

    
    idx = 0
    for x in csv:
        index[idx] = x[0]
        letters[idx] = x[1]
        
        ret[idx] = np.float64(x[2:])
        idx = idx + 1
    
    #print(ret)
    return (header, index, letters, ret)
    
    
def to_f64_try(input):
    try:
        ret =np.float64(input)

        return ret
    except ValueError:
        return input

    
    
    
    
